# Intro to OpenFlow Tutorial with Ryu Controller
<i>Adapted for use with FABRIC from [Intro to OpenFlow Tutorial with Ryu Controller](https://groups.geni.net/geni/wiki/GENIExperimenter/Tutorials/OpenFlowRyu)</i>
    
In this tutorial you will experiment with the OpenFlow routing.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

<br><img src="./figures/OvsTop.png" alt="topology"><br>

<br><br>This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.
#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = "OpenVSSwitch"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

## 2. Guided Experiment

<image src="./figures/SimpleOpenFlow.jpg">

### 2.1. Login to your hosts and preconfigurement
To start our experiment we need to ssh into all of our hosts.
To get ready for the tutorial you will need to have the following windows open:
 - one window with ssh into the controller
 - four windows with ssh into OVS
 - one window with ssh into host1
 - two windows with ssh into host2
 - one window with ssh into host3

### 2.1.a Configure the Software Switch (OVS Window) ¶
1. Login to the OVS host (bridge node)
2. Create an Ethernet bridge that will act as our software switch:

sudo ovs-vsctl add-br br0

3. Prepare the interfaces to be added as ports to the OVS switch
    - Your OVS bridge will be a Layer 2 switch and your ports do not need IP addresses. Before we remove them let's keep some information
        + Run ifconfig
        + Write down the interface names that correspond to the connections to your hosts. The correspondence is
           - Interface with IP 10.10.1.11 to host1 - ethX
           - Interface with IP 10.10.1.12 to host2 - ethY
           - Interface with IP 10.10.1.13 to host3 - ethZ 
    - Remove the IP from your data interfaces.
    - **Be careful not to bring down enp3s0*. This is the control interface, if you bring that interface down you **won't be able to login to your host**. For all interfaces other than eth0 and l0 (your interface names may vary) run :

    <br> ``` sudo ifconfig ethX 0  ```
    <br> ``` sudo ifconfig ethY 0  ```
    <br> ``` sudo ifconfig ethZ 0  ``` 

4. Add all the data interfaces to your switch (bridge).
- **Be careful not to bring down enp3s0*. This is the control interface. The other three interfaces are your data interfaces. (Use the same interfaces as you used in the previous step.)

<br> ``` sudo  ovs-vsctl add-port br0 ethX ```
<br> ``` sudo  ovs-vsctl add-port br0 ethY ```
<br> ``` sudo  ovs-vsctl add-port br0 ethZ ```

5. Trust but verify. Congratulations! You have configured your software switch. To verify the three ports configured run:

<br> ``` sudo ovs-vsctl list-ports br0 ```

### 2.1.b Point your switch to a controller
Note 	*An OpenFlow switch will not forward any packet unless instructed by a controller. Basically the forwarding table is empty, until an external controller inserts forwarding rules. The OpenFlow controller communicates with the switch over the control network and it can be anywhere in the Internet as long as it is reachable by the OVS host.*

1. Login to your controller
2. Find the control interface IP of your controller, use ifconfig and note down the IP address of eth0.
3. In order to point our software OpenFlow switch to the controller, in the ovs terminal window, run:

<br> ``` sudo ovs-vsctl set-controller br0 tcp:<controller_ip>:6633 ```

4. Set your switch to fail-safe-mode. For more info read the standalone vs secure mode section. Run:

<br> ``` sudo ovs-vsctl set-fail-mode br0 secure ```

5. Trust but verify. You can verify your OVS settings by issuing the following:

<br> ``` sudo ovs-vsctl show ```

### 2.1.c standalone vs secure mode

The OpenFlow controller is responsible for setting up all flows on the switch, which means that when the controller is not running there should be no packet switching at all. Depending on the setup of your network, such a behavior might not be desired. It might be best that when the controller is down, the switch should default back to being a learning layer 2 switch. In other circumstances however this might be undesirable. In OVS this is a tunable parameter, called fail-safe-mode which can be set to the following parameters:

    **standalone** [default]: in this case OVS will take responsibility for forwarding the packets if the controller fails
    **secure**: in this case only the controller is responsible for forwarding packets, and if the controller is down all packets are dropped. 

In OVS when the parameter is not set it falls back to the standalone mode. For the purpose of this tutorial we will set the fail-safe-mode to secure, since we want to be the ones controlling the forwarding. 


### 2.2 Use a Learning Switch Controller
In this example we are going to run a very simple learning switch controller to forward traffic between host1 and host2.
1. First start a ping from host1 to host2, which should timeout, since there is no controller running. 
<br>```ping <host2_IP> -c 10```
2. We have installed the Ryu controller under ~/ryu on the controller host. Ryu comes with a set of example modules that you can use out of the box. One of the modules is a learning switch. Start the learning switch controller which is already available by running the following two commands: 
<br>``` cd ~/ryu ```
<br>``` ryu-manager ryu/app/simple_switch.py ```

The output should look like this: 
<br>```    loading app ryu/app/simple_switch.py  ```
<br>```    loading app ryu.controller.ofp_handler  ```
<br>```    instantiating app ryu.controller.ofp_handler of OFPHandler  ```
<br>```    instantiating app ryu/app/simple_switch.py of SimpleSwitch  ```
<br>In the terminal of ```host1```, ping ```host2```:
<br>```    [experimenter@host1 ~]$ ping host2   ```
<br>```    PING host2-lan1 (10.10.1.2) 56(84) bytes of data.  ```
<br>```    From host1-lan0 (10.10.1.1) icmp_seq=2 Destination Host Unreachable  ```
<br>```    From host1-lan0 (10.10.1.1) icmp_seq=3 Destination Host Unreachable  ```
<br>```    From host1-lan0 (10.10.1.1) icmp_seq=4 Destination Host Unreachable  ```
<br>```    64 bytes from host2-lan1 (10.10.1.2): icmp_req=5 ttl=64 time=23.9 ms  ```
<br>```    64 bytes from host2-lan1 (10.10.1.2): icmp_req=6 ttl=64 time=0.717 ms  ```
<br>```    64 bytes from host2-lan1 (10.10.1.2): icmp_req=7 ttl=64 time=0.654 ms  ```
<br>```    64 bytes from host2-lan1 (10.10.1.2): icmp_req=8 ttl=64 time=0.723 ms  ```
<br>```    64 bytes from host2-lan1 (10.10.1.2): icmp_req=9 ttl=64 time=0.596 ms  ```

        Now the ping should work.

    Go to your controller host and take a look at the print outs. You should see that your controller installed flows based on the mac addresses of your packets. 


### 2.3  Look around your OVS switch
1. If you are using OVS, to see the flow table entries on your OVS switch:
<br>```sudo ovs-ofctl dump-flows br0```
<br>You should see at least two table entries: One for ICMP Echo (icmp_type=8) messages from host1 to host2 and one for ICMP Echo Reply (icmp_type=0) messages from host2 to host1. You may also see flow entries for arp packets. 
2. To see messages go between your switch and your controller, open a new ssh window to your controller node and run tcpdump on the eth0 interface and on the tcp port that your controller is listening on usually 6633. (You can also run ```tcpdump``` on the ```OVS``` control interface if you desire.)
<br>```sudo tcpdump -i eth0 tcp port 6633```
<br>You will see (1) periodic keepalive messages being exchanged by the switch and the controller, (2) messages from the switch to the controller (e.g. when there is a table miss) and an ICMP Echo message in, and (3) messages from the controller to the switch (e.g. to install new flow entries). 
3. Kill your Ryu controller by pressing ```Ctrl-C```. 
4. Notice what happens to your ping on host1. 
5. If you are using OVS, check the flow table entries on your switch:
<br>```sudo ovs-ofctl dump-flows br0```
<br>You will see flow table entries in the switch. The entries time to expire value is set to infinity. 
6. Delete the entries on the OVS:
<br>```sudo ovs-ofctl del-flows br0```
<br>Notice what happens to your ping on host1. 

Soft vs Hard Timeouts
All rules on the switch have two different timeouts:
 - **Soft Timeout**: This determines for how long the flow will remain in the forwarding table of the switch if there are no packets received that match the specific flow. As long as packets from that flow are received the flow remains on the flow table.
 - **Hard Timeout**: This determines the total time that a flow will remain at the forwarding table, independent of whether packets that match the flow are received; i.e. the flow will be removed after the hard timeout expires. 

Can you tell now why there were packets flowing even after you killed your controller? 

### 2.4 Download the Ryu apps
To help you get started with your controller writing, we will provide:
 - skeleton files for the controllers where you only need to complete some missing functionality
 - the solution: fully implemented controllers
 - a utility library that makes some of the Ryu messages easier to write 

**In the controller terminal execute:**
<br>``` mkdir ~/ryu/ryu/ext     ```
<br>``` cd ~/ryu/ryu/ext/      ```
<br>``` sudo wget https://github.com/GENI-NSF/geni-tutorials/raw/master/OVSRyu/ryu-intro-ctrlapps.tar.gz      ```
<br>``` sudo tar xvfz ryu-intro-ctrlapps.tar.gz      ```

**Useful Tips for writing your controller**

In order to make this first experience of writing a controller easier, we wrote some helpful functions that will abstract some of the particularities of Ryu away. These functions are located in ```~/ryu/ryu/ext/utils.py```, so while you write your controller consult this file for details.

Functions that are implemented include:

 - packetIsIP : Test if the packet is IP
 - packetIsARP : Test if the packet is ARP
 - packetIsRequestARP : Test if this is an ARP Request packet
 - packetIsReplyARP : Test if this is an ARP Reply packet
 - packetArpDstIp : Test what is the destination IP in an ARP packet
 - packetArpSrcIp : Test what is the source IP in an ARP packet
 - packetIsTCP : Test if a packet is TCP
 - packetDstIp : Test the destination IP of a packet
 - packetSrcIp : Test the source IP of a packet
 - packetDstTCPPort : Test the destination TCP port of a packet
 - packetSrcTCPPort : Test the source TCP port of a packet
 - createOFAction : Create one OpenFlow action
 - getFullMatch : get the full match out of a packet
 - createFlowMod : create a flow mod
 - createArpRequest : Create an Arp Request for a different destination IP
 - createArpReply : Create an Arp Reply for a different source IP 


### 2.5  Debugging your Controller
While you are developing your controller, some useful debugging tools are:
1. Print messages
<br> Run your controller in verbose mode (add --verbose) and add print messages at various places to see what your controller is seeing.

2. Check the status in the switch
<br>If you are using an OVS switch, you can dump information from your switch. For example, to dump the flows:
<br> ```sudo ovs-ofctl dump-flows br0```
<br> Two other useful commands show you the status of your switch:
<br> ```sudo ovs-vsctl show ```
<br> ```sudo ovs-ofctl show br0 ```

3. Use Wireshark to see the OpenFlow messages
<br>Many times it is useful to see the OpenFlow messages being exchanged between your controller and the switch. This will tell you whether the messages that are created by your controller are correct and will allow you to see the details of any errors you might be seeing from the switch. <br>You can use wireshark on both ends of the connection, in hardware switches you have to rely only on the controller view.
<br>The controller host and OVS has wireshark installed, including the openflow dissector. For more information on wireshark you can take a look at the [wireshark wiki](http://wiki.wireshark.org/).
<br> Here we have a simple case of how to use the OpenFlow dissector for wireshark.
<br> If you are on a Linux friendly machine (this includes MACs) open a terminal and ssh to your controller machine using the -Y command line argument, i.e.
<br> ```ssh -Y <username>@<controller>```
<br> Assuming that the public IP address on the controller is eth0, run wireshark by typing:
<br> ```sudo wireshark -i eth0& ```
<br> however we are going to skip this step since we dont have a display.
When the wireshark window pops up, you might still have to choose eth0 for a live capture. And you will want to use a filter to cut down on the chatter in the wireshark window. One such filter might be just seeing what shows up on port 6633. To do that type tcp.port eq 6633 in the filter window, assuming that 6633 is the port that the controller is listening on. 

### 2.6 Run a traffic duplication controller
In the above example we ran a very simple learning switch controller. 

 The power of OpenFlow comes from the fact that you can decide to forward the packet anyway you want based on the supported OpenFlow actions. A very simple but powerful modification you can do, is to duplicate all the traffic of the switch out a specific port. This is very useful for application and network analysis. You can imagine that at the port where you duplicate traffic you connect a device that does analysis. For this tutorial we are going to verify the duplication by doing ```tcpdump``` on two ports on the OVS switch.

 1. **Use the interfaces that are connected to ```host2``` and ```host3```**.
    - Software Switch (OVS): You should have noted down the interfaces for host2 and host3 in section 2.1.a. If you have not noted the interfaces for host2 and host3 down (in section 2a), you can run tcpdump on OVS interfaces to figure out the interfaces for host2 and host3. This will allow you to see all traffic going out the interfaces.
    
<br>To see that duplication is happening, on the OVS host, run:
<br> ```sudo tcpdump -i <data_interface_name_to_host2>   ```
<br> ```sudo tcpdump -i <data_interface_name_to_host3>   ```

You should see traffic from host1 to host2 showing up in the tcpdump window for host3. As a comparison, you will notice that no traffic shows up in that window when the controller is running the learning switch (at /tmp/ryu, type ./bin/ryu-manager ryu/ext/simple_switch.py).

2. **In the controller host directory** ```/tmp/ryu/ryu/ext``` **you should see two files**: 
<br>**i.** **myDuplicateTraffic.py** : This is the file that has instructions about how to complete the missing information. Go ahead and try to implement your first controller.
<br>**ii.** **DuplicateTraffic.py**: This has the actual solution. You can just run this if you don't want to bother with writing a controller.
<br>**iii.** **duplicate.config**: in this file, you need to specify which port you want to duplicate traffic to. We will be duplicating host2 traffic and send it to host3, so put host3 port number here. We noted down port numbers in section 2a. To figure out which port maps to which interface, use can also use "sudo ovs-ofctl show br0" on ovs node. (You can use any editor to edit the file, e.g. use nano by typing "sudo nano duplicate.config") 

3. Run your newly written controller to duplicate the traffic. (update duplicate.config file with host3 port number. We noted it down in section 2a ). You can also use the solution file DuplicateTraffic.py to duplicate traffic:
<br> ```cd ~/ryu ```
<br> ```ryu-manager ryu/ext/myDuplicateTraffic.py ```
<br> *note* if you get errors tryng to run the experiments in the **Ext** dir the utils file might be corrupted, please download the utils.py script [here]() and place it under ~/ryu/ryu/ext, by uploading it to the node or using wget on the directory
4. To test it go to the terminal of host1 and try to ping host2:
<br> ```ping 10.10.1.2 ```
<br> If your controller is working, your packets will register in both terminals running tcpdump. 

5. Stop the Ryu controller using ```Ctrl-C``. 

### 2.7  Run a port forward Controller
Now let's do a slightly more complicated controller. OpenFlow gives you the power to overwrite fields of your packets at the switch, for example the TCP source or destination port and do port forwarding. You can have clients trying to contact a server at port 5000, and the OpenFlow switch can redirect your traffic to a service listening on port 6000.

1. Under the ```/tmp/ryu/ryu/ext``` directory there are two files: **PortForwarding.py** and **myPortForwarding.py** that are similar to the previous exercise. Both of these controllers are configured by a configuration file at ext/port_forward.config. Use ```myPortForwarding.py``` to write your own port forwarding controller. 

2. To test your controller we are going to use netcat. Go to the two terminals of host2. In one terminal run:
<br> ```nc -l 5000```
<br>and in the other terminal run
<br> ```nc -l 6000```

3. Now, start the simple layer 2 forwarding controller. We are doing this to see what happens with a simple controller.
<br> ```cd /tmp/ryu```
<br> ```./bin/ryu-manager ryu/ext/simple_switch.py```

4. Go to the terminal of host1 and connect to host2 at port 5000:
<br> ```nc 10.10.1.2 5000```
5. Type something and you should see it at the terminal of host2 at port 5000. 
6. Now, stop the simple layer 2 forwarding controller by Ctrl-C. 
7. And start your port forwarding controller (if you have written your controller then use myPortForwarding in the following command):
<br> ```./bin/ryu-manager ryu/ext/PortForwarding.py```
8. Repeat the netcat scenario described above. Now, your text should appear on the other terminal of host2 which is listening to port 6000. 
9. Stop your port forwarding controller using ```Ctrl-C```

### 2.8 Run a Server Proxy Controller

As our last exercise, instead of diverting the traffic to a different server running on the same host, we will divert the traffic to a server running on a different host and on a different port.

1. Under the ```/tmp/ryu/ryu/ext/``` directory there are two files: **Proxy.py** and **myProxy.py** that are similar to the previous exercise. Both of these controllers are configured by the configuration file ```proxy.config```. Use myProxy.py to write your own proxy controller. 

2. On the terminal of ```host3``` run a netcat server:

<br> ```nc -l 7000 ```

3. On your controller host, open the /tmp/ryu/ryu/ext/myProxy.py file, and edit it to implement a controller that will divert traffic destined for host2 to host3. Before you start implementing think about what are the side effects of diverting traffic to a different host.
   - Is it enough to just change the IP address?
   - Is it enough to just modify the TCP packets? 

    <br> If you want to see the solution, it's available in file /tmp/ryu/ryu/ext/Proxy.py file.

4. To test your proxy controller run (if you have written your controller then use myProxy in the following command):

<br> ```cd /tmp/ryu ```
<br> ```ryu-manager ryu/ext/Proxy.py ```

5. Go back to the terminal of ```host1``` and try to connect netcat to ```host2``` port 5000

<br> ```nc 10.10.1.2 5000 ```

6. If your controller works correctly, you should see your text showing up on the terminal of ```host3```. 

## 3. Delete your bridge
Before moving to the next step make sure you delete the bridge you have created, especially if you are using the same reservation for a different exercise:
<br> ```sudo ovs-vsctl del-br br0```


## Cleanup Resources
Once you have completed the assignment shut down the slice.

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")